# UCSB DS100 FINAL PROJECT

## Due Date:  June 12, midnight

<span style="color:red">
    <h1> U.S. Educational Finances </h1>
</span>

<span style="color:blue">
<h3> Name: Jason Yu </h3>
</span>

## Abstract

Education is an important foundation for many Americans, and funding for schools is a big aspect of this.  It is not only important that this is crucial for students, but also for the future. Therefore, the revenue and distribution of profit from schools in all 50 states is imperative to address the inequalities of education funding in certain states. In this project, the use of translations, visualization, and linear regression allowed me to conclude that school spending on support services and other beneficial services will not only result in happier students, but also increased revenue. This project shows how important education funding is for the United States. 

## Introduction

The allocation of money circulating in the United States elementary and middle schools affects almost all Americans. Funding education takes part in a huge role in the outcomes of success, for not only the students but also the future. Many people believe that students tend to thrive in better funded schools than less funded schools. Education should be at a level playing field for all students by distributing an equal amount of funding between the 50 states (Biddle 2002). Thus, the amount of given revenue and generated profit is important to determine the changes in specific states that will need to partake, in order to achieve maximum success in United States schools. The dataset used in this analysis contains the financial information surveyed from all public schools across the country. Data includes different categories of revenue, expenditures, number of students enrolled, the state that was surveyed, and the year that the survey was conducted.

### Questions of Interest

- What is a multilinear regression model that can predict total revenue of a school, and how accurate is it?
- Can we use principal component analysis to reduce the number of dimensions while still having an accurate model?
- What is the general trend of school revenue over time? 
- Is there an uneven distribution of profit amongst the schools in the United States?

### Setting Things Up (Packages, Dataset)

In [1]:
# Import Pacakges
import numpy as np
import pandas as pd
from pandas.api.types import CategoricalDtype
import altair as alt
import matplotlib
import matplotlib.pyplot as plt
import sklearn
import os
from sklearn.linear_model import LinearRegression

In [2]:
states = pd.read_csv('states.csv')

## Data and Methods

### EDA

Look over at what the data looks like!

In [3]:
states.head()

,STATE,YEAR,ENROLL,TOTAL_REVENUE,FEDERAL_REVENUE,STATE_REVENUE,LOCAL_REVENUE,TOTAL_EXPENDITURE,INSTRUCTION_EXPENDITURE,SUPPORT_SERVICES_EXPENDITURE,OTHER_EXPENDITURE,CAPITAL_OUTLAY_EXPENDITURE
0,Alabama,1992,NaN,2678885,304177,1659028,715680,2653798,1481703,735036,NaN,174053
1,Alaska,1992,NaN,1049591,106780,720711,222100,972488,498362,350902,NaN,37451
2,Arizona,1992,NaN,3258079,297888,1369815,1590376,3401580,1435908,1007732,NaN,609114
3,Arkansas,1992,NaN,1711959,178571,958785,574603,1743022,964323,483488,NaN,145212
4,California,1992,NaN,26260025,2072470,16546514,7641041,27138832,14358922,8520926,NaN,2044688


- There are 51 missing variables. Enrollment data is missing for all states during the year 1992 so we will remove that year from our data.

In [4]:
states.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1275 entries, 0 to 1274
Data columns (total 12 columns):
STATE                           1275 non-null object
YEAR                            1275 non-null int64
ENROLL                          1224 non-null float64
TOTAL_REVENUE                   1275 non-null int64
FEDERAL_REVENUE                 1275 non-null int64
STATE_REVENUE                   1275 non-null int64
LOCAL_REVENUE                   1275 non-null int64
TOTAL_EXPENDITURE               1275 non-null int64
INSTRUCTION_EXPENDITURE         1275 non-null int64
SUPPORT_SERVICES_EXPENDITURE    1275 non-null int64
OTHER_EXPENDITURE               1224 non-null float64
CAPITAL_OUTLAY_EXPENDITURE      1275 non-null int64
dtypes: float64(2), int64(9), object(1)
memory usage: 119.7+ KB


Here we process the data by removing null rows, making the column names lower case, and adding the profit columns.

In [5]:
def process_data(data):
    # copy of original
    data_copy = data.copy()
    
    # remove null values
    data_copy = data_copy.dropna(axis = 'rows', how='any')
    
    # make columns lower case
    data_copy.columns = data_copy.columns.str.lower()
    
    # add profit column
    data_copy['profit'] = data_copy['total_revenue'] - data_copy['total_expenditure']
    
    # add profitable column
    data_copy['profitable'] = data_copy['profit'] >= 0
    
    return data_copy

In [6]:
states_clean = process_data(states)

In [7]:
states_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1224 entries, 51 to 1274
Data columns (total 14 columns):
state                           1224 non-null object
year                            1224 non-null int64
enroll                          1224 non-null float64
total_revenue                   1224 non-null int64
federal_revenue                 1224 non-null int64
state_revenue                   1224 non-null int64
local_revenue                   1224 non-null int64
total_expenditure               1224 non-null int64
instruction_expenditure         1224 non-null int64
support_services_expenditure    1224 non-null int64
other_expenditure               1224 non-null float64
capital_outlay_expenditure      1224 non-null int64
profit                          1224 non-null int64
profitable                      1224 non-null bool
dtypes: bool(1), float64(2), int64(10), object(1)
memory usage: 135.1+ KB


#### Total Revenue

We will examine a density plot of our target variable `total_revenue`

In [8]:
alt.Chart(states_clean).transform_density(
    'total_revenue',
    as_=['total_revenue', 'density']
).mark_area().encode(
    x=alt.X('total_revenue:Q', title='Total Revenue'),
    y=alt.Y('density:Q', title='Density')
).properties(
    title = 'Distribution of Total Revenue'
).configure_axis(
    labelFontSize = 14,
    titleFontSize = 16
).configure_legend(
    labelFontSize = 13,
    titleFontSize = 14
)

alt.Chart(...)

In [9]:
states_clean.describe()

,year,enroll,total_revenue,federal_revenue,state_revenue,local_revenue,total_expenditure,instruction_expenditure,support_services_expenditure,other_expenditure,capital_outlay_expenditure,profit
count,1224.000000,1.224000e+03,1.224000e+03,1.224000e+03,1.224000e+03,1.224000e+03,1.224000e+03,1.224000e+03,1.224000e+03,1.224000e+03,1.224000e+03,1.224000e+03
mean,2004.500000,9.175416e+05,9.290765e+06,7.873945e+05,4.312719e+06,4.190651e+06,9.395936e+06,4.864428e+06,2.737271e+06,4.299509e+05,9.242366e+05,-1.051710e+05
std,6.925016,1.066514e+06,1.191733e+07,1.164312e+06,5.620372e+06,5.564716e+06,1.215400e+07,6.385360e+06,3.399931e+06,5.347893e+05,1.349417e+06,5.841502e+05
min,1993.000000,4.386600e+04,4.656500e+05,3.367200e+04,0.000000e+00,2.391700e+04,4.816650e+05,2.655490e+05,1.399630e+05,1.154100e+04,1.270800e+04,-5.487742e+06
25%,1998.750000,2.645145e+05,2.224650e+06,1.931018e+05,1.191590e+06,7.380178e+05,2.211494e+06,1.195616e+06,6.527800e+05,1.034492e+05,1.886915e+05,-1.845112e+05
50%,2004.500000,6.499335e+05,5.256748e+06,4.219105e+05,2.614030e+06,2.098524e+06,5.415694e+06,2.737071e+06,1.567025e+06,2.717040e+05,5.295925e+05,-1.959600e+04
75%,2010.250000,1.010532e+06,1.110987e+07,8.479270e+05,5.224320e+06,4.793464e+06,1.089286e+07,5.648436e+06,3.308660e+06,5.172222e+05,9.908930e+05,5.645475e+04
max,2016.000000,6.307022e+06,8.921726e+07,9.990221e+06,5.090457e+07,3.610526e+07,8.532013e+07,4.396452e+07,2.605802e+07,3.995951e+06,1.022366e+07,3.897129e+06


There does not appear to be any significant outliers that we will have to remove or deal with.

Now we will take a look at the pairs plot to observe any relationships between the major variables.

In [10]:
alt.Chart(states_clean).mark_point().encode(
    alt.X(alt.repeat('column'), type = 'quantitative'),
    alt.Y(alt.repeat('row'), type = 'quantitative'),
).properties(
    width = 100,
    height = 100
).repeat(
    row = ['year', 'enroll', 'total_revenue', 'total_expenditure'],
    column = ['year', 'enroll', 'total_revenue', 'total_expenditure']
)

alt.RepeatChart(...)

Year has no real linear relationship with any other of the predictors. Enroll has a somewhat, but not strong linear relationship with total_revenue and total_expenditure. The most glaring observation is that total_revenue and total_expenditure have a very strong linear relationship.

#### PCA

We will now use PCA to see if we can identify any other interesting patterns.

In [11]:
states_pca = states_clean.drop('state', axis=1).reset_index(drop=True)

from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

# Initialize packages
scaler = StandardScaler()
pca = PCA(n_components=4)

# Scale Data and Reduce Components
x_scaled = scaler.fit_transform(states_pca)
pca.fit(x_scaled)

# Create dataframe with first two PC's
components = (pca.fit_transform(x_scaled) @ pca.components_)
df_components = pd.DataFrame({
    'pc1': components[:,0],
    'pc2': components[:,1]
})

alt.Chart(df_components).mark_point().encode(
    x = "pc1",
    y = "pc2"
)

alt.Chart(...)

We can observe some dintinct patterns of clustering. This could grouped by the year of each observation.

## Research Question 1: What is a multilinear regression model that can predict total revenue of a school, and how accurate is it?

In [12]:
# Split dataset into test and training
def generate_training_sample(data, frac_training=0.5, replace=False):

    full_data_len = len(data)
    train_indices = np.random.choice(full_data_len, np.int(frac_training*full_data_len), replace=replace)
    
    # Create train and test by indexing into `full_data`
    train = data.iloc[train_indices,:]
    test = data.loc[data.index.difference(train_indices)]
    test.drop('total_revenue', axis=1, inplace=True)
    return train, test
    
    
training_data, test_data = generate_training_sample(states_clean)

In [13]:
# Helper function for process_data_gm
def select_columns(data, columns):
    cols = [c for c in columns if c in data.columns]
    return data.loc[:, cols]

# Function to split data in X and Y
def process_data_gm(data, columns):    
    # Transform Data, Select Features
    data = select_columns(data, columns)
    
    # Return predictors and response variables separately
    if 'total_revenue' in data.columns:
        X = data.drop(['total_revenue'], axis = 1) 
        y = data.loc[:, 'total_revenue']
    elif 'profit' in data.columns:
        X = data.drop(['profit'], axis = 1) 
        y = data.loc[:, 'profit']
    else:
        # Return none for y if SalePrice is not known
        X = data
        y = None
    
    return X, y

In [14]:
columns = ['total_revenue', 'year', 'enroll','instruction_expenditure',
           'support_services_expenditure','other_expenditure',
           'capital_outlay_expenditure']

X_train, y_train = process_data_gm(training_data, columns)

X_test, _ = process_data_gm(test_data, columns)

In [15]:
linear_model = LinearRegression()

## Fit the linear model
linear_model.fit(X = X_train, y = y_train)

## Generate predictions
y_fitted = linear_model.predict(X_train)
y_predicted = linear_model.predict(X_test)

## Coefficients in the linear regression model
coefs = linear_model.coef_
print(coefs)

[ 4.18177375e+02 -1.72564668e-01  8.45740132e-01  1.57311582e+00
  1.88589150e+00  3.92037085e-01]


In [16]:
X_train.head(1)

,year,enroll,instruction_expenditure,support_services_expenditure,other_expenditure,capital_outlay_expenditure
326,1998,830744.0,3584678,1902782,302974.0,558202


Taking a look at the model's coefficients, `support_services_expenditure`, `other_expenditure`, and `year` seem to be indicators for the increase of total_revenue.

In [17]:
coefs[3]*100000

157311.58222806948

A 100000 increase in support_services_expenditure should increase total revenue by about 162918.  

### Model Checking

In [18]:
### RMSE
def rmse(actual, predicted):
    """
    Calculates RMSE from actual and predicted values
    Input:
      actual (1D array): vector of actual values
      predicted (1D array): vector of predicted/fitted values
    Output:
      a float, the root-mean square error
    """
    x = np.sum((actual - predicted)**2) / len(actual)
    return np.sqrt(x)

def mse(y, y_hat):
    return np.mean((y-y_hat)**2)
    
loss1 = rmse(y_train, y_fitted)
loss1

499374.07353731565

In [19]:
# Residual Plot
tidy_df = training_data.copy()
tidy_df['Residual Total Revenue'] = y_train-y_fitted

alt.Chart(tidy_df).mark_point().encode(
    y = alt.Y('total_revenue', title='Total Revenue'),
    x = alt.X('Residual Total Revenue')
).properties(
    title='Total Revenue vs Residual Total Revenue'
).configure_title(fontSize = 18).configure_axis(
    labelFontSize = 14,
    titleFontSize = 16
)

alt.Chart(...)

From this residual plot we can say that the model fits our data fairly well. The majority of points are clustered around 0 for Residual Total Expenditure and we do not see too many points straying away. Our model does pretty well even for schools with very high revenue.

## Research Question 2: Can we use principal component analysis to reduce the number of dimensions while still having an accurate model?

In [20]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

# Initialize packages
scaler = StandardScaler()
pca = PCA(n_components=4)

# Scale Data and Reduce Components
x_scaled = scaler.fit_transform(X_train)
pca.fit(x_scaled)
x_scaled = pca.fit_transform(x_scaled)

# Fit linear model
linear_model = LinearRegression()
linear_model.fit(X = x_scaled, y = y_train)

y_fitted_new = linear_model.predict(x_scaled)

loss2 = rmse(y_train, y_fitted_new)

In [21]:
print(loss2 <= 1.1*loss1)

True


We only need 4 principal components in order to get a loss value within 10% of our orignal loss value.

In [22]:
pca.explained_variance_ratio_

array([0.79591303, 0.16285598, 0.02111025, 0.01271599])

We can also see that the first two principal components explain over 90% of the variance in our predictors.

In [23]:
explained_var1 = pd.DataFrame({
    'PC #': range(1,5), 
    'Fraction of Variance Explained' : pca.explained_variance_ratio_
})

# Draw your Altair visualization
alt.Chart(explained_var1, 
          title="Variance Explained by Principal Components"
).mark_bar(size=30).encode(
    alt.X('PC #:O'),
    alt.Y('Fraction of Variance Explained:Q')
).configure_title(
    fontSize = 18
).configure_axis(
    labelAngle=0,
    labelFontSize = 14,
    titleFontSize = 16
).properties(width=500)

alt.Chart(...)

In [24]:
tidy_df = training_data.copy()
tidy_df['Residual Total Revenue'] = y_train-y_fitted_new

alt.Chart(tidy_df).mark_point().encode(
    y = alt.Y('total_revenue', title='Total Revenue'),
    x = alt.X('Residual Total Revenue')
).properties(
    title='Total Revenue vs Residual Total Revenue'
).configure_title(fontSize = 18).configure_axis(
    labelFontSize = 14,
    titleFontSize = 16
)

alt.Chart(...)

After regression on the reduced model, most points are still fairly closely clustered to the 0 residual threshold. The plot also significantly resembles the residual plot of the original model. Therefore, we can say that it is possible to reduce the dimensions of the model to 4 components while maintaining an accurate model.

## Research Question 3: What is the general trend of school revenue over time?

In [25]:
subset_head = list(states_clean.groupby(['state'])\
        .agg({'total_revenue':np.sum})\
        .sort_values('total_revenue', ascending=False).head(10).index)

subset_tail = list(states_clean.groupby(['state'])\
        .agg({'total_revenue':np.sum})\
        .sort_values('total_revenue').head(20).index)

head_df = states_clean.loc[states_clean['state'].isin(subset_head)]
tail_df = states_clean.loc[states_clean['state'].isin(subset_tail)]

In [26]:
top10 = alt.Chart(head_df).mark_line().encode(
    x = alt.X('year:O', title='Year'),
    y = alt.Y('total_revenue:Q', title= 'Total Revenue'),
    color = 'state:N').properties(
    title='Total Revenue Over Time',
    width=250
)

bottom20 = alt.Chart(tail_df).mark_line().encode(
    x = alt.X('year:O', title='Year'),
    y = alt.Y('total_revenue:Q', title= 'Total Revenue'),
    color = 'state:N').properties(
    title='Total Revenue Over Time',
    width = 250
)

alt.hconcat(top10,bottom20).configure_title(
    fontSize=18
).configure_axis(
    labelFontSize = 14,
    titleFontSize = 16
).configure_legend(
    labelFontSize = 13,
    titleFontSize = 14
)

alt.HConcatChart(...)

In general, the total revenue of schools increases over time. Revenue slightly dipped during the time period of 2008-2009 due to recession but overall sees a steady increase over time.  

## Research Question 4: Is there an uneven distribution of profit amongst the schools in the United States?

In [27]:
alt.Chart(states_clean).transform_density(
    'profit',
    as_=['profit', 'density']
).mark_area().encode(
    x=alt.X('profit:Q', title='Profit'),
    y=alt.Y('density:Q', title='Density')
).properties(
    title='Distribution of Profit'
).configure_axis(
    labelFontSize = 14,
    titleFontSize = 16
).configure_legend(
    labelFontSize = 13,
    titleFontSize = 14
).configure_title(fontSize=18)

alt.Chart(...)

The distribution of profit between schools seems well distributed. Most schools hover around 0 profit. Also, there are more schools that net negative profit than positive, which makes sense as schools are not meant to be profitable.

In conclusion, our analysis suggests a school’s total revenue is best estimated by its allocation of spending. We learned that the number of students that attend a school is not a  very significant predictor of how much funding the school receives. From the linear regression, the other spending category provided substantial increases in total revenue. Schools that wish to learn more about ways to improve revenue may look into investigating which of the other expenditure categories are most effective. These could include spending more on food, business operations, or other forms. In addition, spending more on support services could also be a good way to improve their revenue. We also identified that profits are evenly distributed amongst schools but there are still more schools with negative profit that positive profit.  

We originally attempted to predict whether a school was profitable or not but could not succeed with the data set given. Schools that were profitable had almost completely overlapping spending patterns as those that were not
We believe the results from this analysis are able to be trusted for multiple reasons. First, the data is collected with a solid process, leading to minimal biases. Also, the conclusions reached at each question can be supported with real world experiences and logical reasoning.  

We believe the results from this analysis are able to be trusted for multiple reasons. First, the data is collected with a solid process, leading to minimal biases. Also, the conclusions reached at each question can be supported with real world experiences and logical reasoning.


# Citations

- data:
https://www.kaggle.com/noriuk/us-educational-finances#states.csv
- lab8:
Used Linear Regression fitting code
- lab9:
Used PCA code
- hw3:
Used pairs plot code
- hw4:
Used code for scree plot
- hw5:
Used provided methods for processing data, calculating loss function, and generating random sample

- Website used for Introduction:
http://www.ascd.org/publications/educational-leadership/may02/vol59/num08/Unequal-School-Funding-in-the-United-States.aspx
